# Lab 6 Clasificadores de la distancia mínima y 1NN

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut

In [2]:
iris = pd.read_csv('iris_clean.csv')
creditCard = pd.read_csv('creditCard_clean.csv')
obesity = pd.read_csv('obesity_clean.csv')

In [3]:
featuresIris = iris[['sepal length','sepal width','petal length','petal width','class_encoded']]
featuresCredit = creditCard[['Debt','Married','Industry_encoded','YearsEmployed','PriorDefault','Employed','CreditScore','Approved']]
featureObesity = obesity[['Height','Weight','BMI','ObesityCategory_encoded']]

## Metodos de validacion

### Holdout

In [4]:
def holdout_validation(X,y,r):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y,random_state=42)
    return X_train, X_test, y_train, y_test

### Fold Cross-Validation

In [5]:
def k_fold_cross_validation(X, y, K=10):
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
    
    folds = []

    for train_index, test_index in skf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        folds.append((X_train, X_test, y_train, y_test))
    
    return folds

### Leave-One-Out.

In [6]:
def leave_one_out_validation(X, y):
    loo = LeaveOneOut()
    leaveVali = []
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        leaveVali.append((X_train, X_test, y_train, y_test))

    return leaveVali

## Clasificador de distancia minima

In [7]:
# Aqui va la logica de la distancia minima sin biblioteca

## Clasificador 1NN

In [8]:
# Aqui va el algoritmo de 1NN